# Ce fichier est la version en cours du script de pré-processing du corpus d'entraînement, avant annotation. Il vise à prendre en entrée l'ensemble des décrets d'urgences, les transformer en docs spacy, et les faire ressortir en JSONL pour les corriger dans Doccano.  Il comprend également un script de détection des erreurs de datation dans les XML (passés en dataframe) et créé 10 sous-groupes des décrets d'urgences pour l'annotation sur doccano online (heroku).

## imports et initialisations

In [1]:
import spacy
import json
import codecs
import pandas as pd
import random
import re
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex

In [2]:
structures_adm = ["commune", "Commune", "district", "Département", "département", "départemens", "Départemens", "municipalité", "Municipalité", "ville", "village", "canton"
         ]
structures_adm2 = ["commune", "Commune", "district", "Département", "département", "départemens", "Départemens", "municipalité", "Municipalité", "ville", "village", "canton"
         ]
lieux = ['Basse-Alpes', 'Maine-&-Loire', 'Seine', 'Alpes-maritimes', 'Arriége', 'Charente-inférieure', 'Côte-d\'or', 'Côtes-du-Nord', 'Dyle', 'Escaut', 'Finistère', 'Forêts', 'Haute-Garonne', 'Ille-et-vilaine', 'Indre-et-Loire', 'Jemmapes', 'Loire-inférieure', 'Lys', 'Meurthe', 'Meuse-inférieure', 'Deux-Nèthes', 'Ourthe', 'Puy-de-Dôme', 'Basses-Pyrénées', 'Pyrénées-Orientales', 'Sambre-et-Meuse', 'Haute-Saone', 'Saone-et-Loire', 'Seine-inférieure', 'Deux-Sèvres', 'La Souterraine', 'Alès', 'Moulins-Engilbert', 'Segré', 'La Charité', 'Nérac', 'Bitche', 'La Ferté-Bernard', 'Toulon', 'Pont-de-Vaux', 'Châtellerault', 'Soissons', 'Noyon', 'Chartres', 'Bourbonne-les-Bains', 'Crépy', 'Autun', 'Mauléon', 'Mirande', 'Tartas', 'Douai', 'Poitiers', 'Clamecy', 'Mirecourt', 'Vihiers', 'Nogent-Le-Rotrou', 'Janville', 'Céret', 'Nevers', 'Digne', 'Mer', 'Saint-Claude', 'Brest', "L'Isle-Jourdain", 'Saint-Gaudens', 'Pontoise', 'Montlieu', 'Limoux', 'Strasbourg', 'Embrun', 'Fougères', 'Monflanquin', 'Calais', 'Provins', 'Montdidier', 'Saint-Marcellin', 'Bourges', 'Falaise', 'Vico', 'Caen', 'Barcelonnette', 'Fresnay-sur-Sarthes', 'Mur-de-Barrez', 'Saint-Rambert', 'Tulle', 'Arcis-sur-Aube', 'Chauny', 'La Rochefoucauld', 'Evreux', 'Vesoul', "Pont-l'Evêque", 'Crest', 'Vendôme', 'Loches', 'La Guerche', 'Mont-de-Marsan', 'Montmorillon', 'Corbeil', 'Avesnes', 'Villefort', 'Neuville-aux-Bois', 'Grand-Andelys', 'Cany', 'Guérêt', "Saint-Jean-d'Angély", 'Bazas', 'Nogaro', 'Breteuil', 'Amiens', 'Altkirch', 'Bellême', 'Melun', 'Commercy', 'Cervione', 'Lamarche', 'Oloron', 'Rodez', 'Saint-Céré AND Figeac', 'Brignoles', 'Argelès', 'Lisieux', 'Barbezieux', 'Amboise', 'Château-Thierry', 'Trévoux', 'Lesparre', 'Apt', 'Saint-Florentin', 'Meyrueis', 'Broons', 'Châteaudun', 'Châtillon-sur-Seine', 'Charolles', 'Thouars', 'Bergerac', 'Lure', 'Monistrol', 'Tonneins', 'Le Blanc', 'Mamers', 'Château-du-Loir', 'Blamont', 'Cognac', 'Billom', 'Gex', 'Laon', 'Etampes', 'Tournon Le Mézenc', 'Le Faouët', 'Dinan', 'Loudéac', 'Bruyères', 'Saint-Flour', 'Sens', 'Wissembourg', 'Challans', 'Saint-Chély', 'Bellac', 'Jussey', 'Paris', 'Ajaccio', 'Pithiviers', 'Bar-le-Duc', 'Cadillac', 'Joyeuse La Tanargue', 'Verdun', 'Carentan', 'Lagrasse', 'Loudun', 'Sancoins', 'Redon', 'Bourg', 'Ploërmel', 'Cosne', 'Valenciennes', 'Saint-Pol', 'Serres', 'Riom', 'Montmarault', 'Bellay', 'Avallon', 'Vire', 'Montivilliers', 'La Porta', 'Romorantin', 'Grandpré', 'Saint-Leonard', 'Cérilly', 'Bain', 'Sancerre', 'Ruffec', 'Alençon', 'Montaigu', 'La Roche-Bernard', 'Montbrisson', 'Roanne', 'Bar-sur-Aube', 'Tallano', 'Bernay', 'Remiremont', 'Saint-Denis', 'Hazebrouck', 'Gap', 'Confolens', 'Châlons', 'Tours', 'Langogne', 'Evaux', 'Perpignan', 'Ernée', 'Benfeld', 'Montreuil', 'Sedan', 'Langeais', 'Aubusson', 'La-Barthe-de-Neste', 'Béthune', 'Gaillac', 'Saint-Paul', 'Sarlat', 'Figeac', 'Florac', 'Ribérac', 'Dieuze', 'Bar-sur-Seine', 'Aurillac', 'Mirepoix', 'Mondoubleau', 'Josselin', 'Saint-Aignan', 'Dieppe', 'Ussel', 'Aubigny', 'Chateau-Renault', 'Machecoul', 'Gray', 'Versailles', 'Grasse', 'Libourne', 'Châteauroux', 'Castres', 'Marcigny', 'Arles', 'Marennes', 'Lectoure', 'Saint-Maximin', 'Narbonne', 'Saint-Affrique', 'Caudebec', 'Excideuil', 'Cholet', 'Lauzun', 'Belvès', 'Pont-Audemer', 'Saint-Yrieix', 'Rethel', 'Romans', 'Besse', 'Montluel', 'Coutances', 'Darney', 'Epinal', 'Dreux', 'Chateaubriant', 'Privas', 'Guérande', 'Orleans', 'Château-Salins', 'Moulins', 'La Rochelle', 'Saint-Dizier', 'Angers', 'Argenton', 'Saint-Germain', 'Carhaix', 'Mende', 'Ile-Rousse', 'Corte', 'Orthez', 'Vaucouleurs', 'Lacaune', 'Clermont-en-Argonne', 'Saint-Palais', "Saint-Geniez-de-Rive-d'Olt", 'Chateauneuf', 'Baugé', 'Bourganeuf', 'Château-Chinon', 'Vierzon', 'Muret', 'Luxeuil', 'Sillé-le-Guillaume', 'Bourg-en-Bresse', 'Bourmont', 'Rozay', 'Lavaur', 'Charleville', 'Beaucaire', 'Sisteron', 'Pont-à-Mousson', 'Dax', 'Fréjus', 'Is-sur-Tille', 'Bordeaux', 'Belfort', 'Saint-Amand', 'Savenay', 'Boulogne', 'Lusignan', 'Châteaumeillant', 'Pons', 'Nemours', 'Saint-Calais', 'Colmar', 'Montauban', 'La-Tour-du-Pin', 'Châtillon-sur-Indre', 'Marseille', 'Grenade', 'Marmande', 'Saint-Florent-le-Vieil', 'Sauveterre', 'Nogent-sur-Seine', 'Joigny', 'Montargis', 'Nyons', 'Arnay-le-Duc', 'Vitry-le-François', 'Pont Saint-Esprit', 'Saint-Junien', 'Draguignan', 'Casteljaloux', 'Montignac', 'Stenay', 'Lunéville', 'Montluçon', 'Melle', 'Clisson', 'Sablé', 'Lauzerte', 'Saint-Maixent', 'Brive-la-Gaillarde', 'Joinville', 'Châtillon-sur-Saône', 'Tarbes', 'Neufchâteau', 'Domfront', 'Briey', 'Castellane', 'Gournay', 'Castelsarrasin', 'Le Puy', 'Mauriac', 'Vannes', 'Paimbeuf', 'Montélimar', 'Ustarritz', 'Mâcon', 'Nimes', 'Agen', 'Saint-Quentin', 'Murat', 'Quingey', 'Dijon', 'Louviers', 'Vienne', 'Chaumont', 'Etain', 'Saint-Omer', 'Cherbourg', 'Sainte-Sever', 'Auxerre', 'Le Donjon', 'Mussidan', 'Dol', 'La Flèche', 'Bourbon-Lancy', 'Barjols', 'Epernay', "Villeneuve-d'Agen", 'Aubin', 'Vitré', 'Prades', 'Pau', 'Châteauneuf', 'Tonnerre', 'Poligny', 'Doullens', 'Vervins', 'Foix', 'Rennes', 'Château-Gontier', 'Blain', 'Châteaulin', 'Le Quesnoy', 'Revel', 'Thionville', 'Orgelet', 'Nantes', 'Millau', 'Haguenau', 'Parthenay', 'Saint-Brieuc', 'Issoudun', 'Blois', 'Toulouse', 'Le Dorat', 'Langres', 'Metz', "L'Aigle", 'Sarrelouis', 'Saint-Jean-de-Losne', 'Bourg-la-Reine', 'Vouziers', 'Sommières', 'Saint-Céré', 'La Châtaigneraie', 'Pontrieux', 'Lamballe', 'Rouen', 'Saint-Hippolyte', 'Mortain', 'Uzès', 'Evron', 'Die', 'Bastia', 'Issoire', 'Uzerche', 'Sainte-Menehould', 'Boussac', 'Nontron', 'Brioude', 'Le Mans', 'Bapaume', 'Saumur', 'Besançon', 'Saint-Mihiel', 'Chinon', 'Bayeux', 'Saintes', 'Decize', 'Fontenay-le-Comte', 'Péronne', 'Valognes', 'Saint-Girons', 'Arbois', 'Grandvilliers', 'Rostrenen', 'Verneuil', 'Lesneven', 'Cahors', 'Condom', 'Saint-Lô', 'Cusset', 'Limoges', 'Toul', 'Lyon', 'Avranches', 'Thiers', 'Corbigny', 'Felletin', 'Pontecroix', 'Beaugency', 'Lille', 'Auray', 'Longwy', 'Vic-Bigorre', 'Ornans', 'Nancy', 'Saint-Fargeau', 'Béziers', 'Montaigut', 'Morlaix', 'Clermont', 'Reims', 'Neufchâtel', 'Hennebont', 'Salon-de-Provence', 'Saint-Malo', 'Ancenis', 'Saint-Pons-de-Thomières', 'Semur-en-Auxois', 'Abbeville', 'Pontivy', 'Gien', 'Argentan', 'Rocroi', 'Briançon', 'Castelnaudary', 'Auch', 'Beauvais', 'Tarascon', 'Saint-Pierre-le-Moûtier', 'Vézelise', 'Mantes', 'Gomesse', 'Châlon-sur-Saône', 'Meaux', 'Angoulême', "Les Sables-d'Olonne", 'Lons-le-Saunier', 'Arras', 'Mortagne', 'Aix-en-Provence', 'Séverac-le-Château', 'Lannion', 'Montpellier', 'Quillan', 'Boulay', 'Clermont-Ferrand', 'Le vigan', 'Dourdan', 'Carcassonne', 'Sézanne', 'Quimper', 'Laval', 'La Roche-sur-Yon', 'Rieux', 'Compiègne', 'Louhans', 'Preuilly', 'Civray', 'Landerneau', 'Rambervilliers', 'Villaines', 'Sarreguemines', 'Baume-les-Dames', 'Champlitte', 'Gourdon', 'Boiscommun', 'Saint-Etienne', 'Nantua', 'Ambert', 'La Châtre', 'Bergues', 'La Réole', 'Beaune', 'Niort', 'Gannat', 'Orange', 'Oletta', 'Hyères', 'Pontarlier', 'Guimgamp', 'Quimperlé', 'Cambrai', 'Albi', 'Mayenne', 'Bagnères-de-Bigorre', 'Saint-Giirons', 'Dôle', 'Troyes', 'Châtillon-les-Dombes', 'Craon', 'Marvejols', 'Périgueux', 'Forcalquier', 'Ervy', 'Saint-Dié', 'Grenoble', 'Morhange', 'Sarrebourg', 'Senlis', 'Lodève', 'Montfort', 'Rochefort', 'Villefranche', 'Valence', 'Montbrison', 'Montélimard', 'Menton', 'Delémont', 'Senones', 'Lodèye', 'Saint-Jean-de-Maurienne', 'rocroi', 'Montbéliard', 'Chambéry', 'Carouge', 'Mézières', 'Moutiers', 'Thonon', 'Avignon', 'Carpentras', 'Neu-Saarwerden', 'Neu-Sarrewerden', 'Puget-Théniers', 'Porrentruy', 'Cluses', 'Nice', 'Annecy', 'Creuse', 'Gard', 'Nièvre', 'Maine-et-Loire', 'Lot-et-Garonne', 'Moselle', 'Sarthe', 'Var', 'Ain', 'Aisne', 'Oise', 'Eure-et-Loir', 'Haute-Marne', 'Saône-et-Loire', 'Pyrénées-Atlantiques', 'Gers', 'Landes', 'Nord', 'Vosges', 'Pyrénées-Orientales', 'Alpes-de-Haute-Provence', 'Loir-et-Cher', 'Jura', 'Finistère', 'Haute-Garonne', 'Seine-et-Oise', 'Charente-Maritime', 'Aude', 'Bas-Rhin', 'Hautes-Alpes', 'Ille-et-Vilaine', 'Pas-de-Calais', 'Seine-et-Marne', 'Somme', 'Isère', 'Cher', 'Calvados', 'Corse', 'Aveyron', 'Corrèze', 'Aube', 'Charente', 'Eure', 'Haute-Saône', 'Drôme', 'Indre-et-Loire', 'Lozère', 'Loiret', 'Seine-Maritime', 'Gironde', 'Haut-Rhin', 'Orne', 'Meuse', 'Lot', 'Hautes-Pyrénées', 'Bouches-du-Rhône', 'Yonne', "Côtes-d'Armor", "Côte-d'Or", 'Deux-Sèvres', 'Dordogne', 'Haute-Loire', 'Indre', 'Meurthe-et-Moselle', 'Puy-De-Dôme', 'Ardèche', 'Morbihan', 'Cantal', 'Vendée', 'Haute-Vienne', 'Manche', 'Allier', 'Ardennes', 'Rhône-et-Loire', 'Marne', 'Tarn', 'Alpes-Maritimes','Alpes Maritimes', 'Ariège', 'Loire-Atlantique', 'Doubs', 'Hérault', 'Drome', 'Loire', 'Golo', 'Vaucluse', 'Mont-Terrible', 'Liamone', 'Mont-Blanc', 'Rhône', 'Seine & Oise', 'Seine & Marne', 'Seine-Inférieure', 'Seine - Inférieure','Seine- Inférieure', 'Léman', 'Côtes-du-Nord'
         ]
structures_locales = ["canton", "administration municipale", "assemblée communale"]
patt = []

## Création de l'entity-ruler

In [3]:
#lemme-ADP-lemme
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_adp_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "ADP"}, 
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_adp_lemme)

#lemme-DET-lemme
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_det_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "DET"}, 
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_det_lemme)
        
#lemme-ADP-DET-lemme
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_adp_det_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "ADP"},  
                 {"POS": "DET"},
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_adp_det_lemme)
        
#lemme-DET-ADP-lemme
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_det_adp_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "DET"},  
                 {"POS": "ADP"},
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_det_adp_lemme)

#lemme-DET-DET-lemme
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_det_det_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "DET"},  
                 {"POS": "DET"},
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_det_det_lemme)

#Lemme-ADP-ADP-lemme
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_adp_adp_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "ADP"},  
                 {"POS": "ADP"},
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_adp_adp_lemme)

#structures_locales-ADP-nom propre
for adm in structures_locales:
    patt_structures_locales_adp = {
        "label": "GPE",
        "pattern": [
            {"TEXT": adm},
            {"POS": "ADP"},
            {"POS": "PROPN"},
        ],
    }
    patt.append(patt_structures_locales_adp)

#structures_locales_DET-nom propre
for adm in structures_locales:
    patt_structures_locales_det = {
        "label": "GPE",
        "pattern": [
            {"TEXT": adm},
            {"POS": "DET"},
            {"POS": "PROPN"},
        ],
    }
    patt.append(patt_structures_locales_det)

#structures_locales_ADP_DET_ nom propre
for adm in structures_locales:
    patt_structures_locales_adp_det = {
        "label": "GPE",
        "pattern": [
            {"TEXT": adm},
            {"POS": "ADP"},
            {"POS": "DET"},
            {"POS": "PROPN"},
        ],
    }
    patt.append(patt_structures_locales_adp_det)

#structures_locales_ADP_ADP_nom_propre
for adm in structures_locales:
    patt_structures_locales_adp_adp = {
        "label": "GPE",
        "pattern": [
            {"TEXT": adm},
            {"POS": "ADP"},
            {"POS": "ADP"},
            {"POS": "PROPN"},
        ],
    }
    patt.append(patt_structures_locales_adp_adp)

#structures_locales_DET_ADP_nom_propre
for adm in structures_adm:
    for lieu in lieux:
        patt_lemme_det_adp_lemme = {
                "label": "GPE", 
                "pattern": [
                 {"TEXT": adm},
                 {"POS": "DET"},  
                 {"POS": "ADP"},
                 {"TEXT": lieu},
                ],
            }
        patt.append(patt_lemme_det_adp_lemme)

#structures_locales_DET_DET_nom_propre
for adm in structures_locales:
    patt_structures_locales_det_det = {
        "label": "GPE",
        "pattern": [
            {"TEXT": adm},
            {"POS": "DET"},
            {"POS": "DET"},
            {"POS": "PROPN"},
        ],
    }
    patt.append(patt_structures_locales_det_det)

#inversion_du_sujet_DET
for adm in structures_adm:
    for adm2 in structures_adm2:
        patt_inversion_du_sujet1 = {
            "label": "GPE",
            "pattern": [
            {"POS": "PROPN"},
            {"TEXT": adm},
            {"POS": "DET"},
            {"TEXT": adm2},
            ],
        }
        patt.append(patt_inversion_du_sujet1)
        
#inversion_du_sujet_ADP
for adm in structures_adm:
    for adm2 in structures_adm2:
        patt_inversion_du_sujet2 = {
            "label": "GPE",
            "pattern": [
            {"POS": "PROPN"},
            {"TEXT": adm},
            {"POS": "ADP"},
            {"TEXT": adm2},
            ],
        }
        patt.append(patt_inversion_du_sujet2)

In [4]:
nlp = spacy.load("fr_core_news_md")

#Ajout d'une règle spéciale pour les traits d'union (Hyphens)
infixes = nlp.Defaults.prefixes + ([r"[./]", r"[-]~", r"[&]~", r"(.'.)"])
infix_re = spacy.util.compile_infix_regex(infixes)

def custom_tokenizer(nlp):
    return Tokenizer(nlp.vocab, infix_finditer=infix_re.finditer)

nlp.tokenizer = custom_tokenizer(nlp)

#Déplacement de l'entity ruler avant la NER (priorisation de l'entity ruler)
nlp.remove_pipe("ner")
ruler = nlp.add_pipe("entity_ruler")
ruler.add_patterns(patt)
nlp.add_pipe("ner", source=spacy.load("fr_core_news_md"))

# Tri des erreurs dans les dates a corriger

In [2]:
corpus_urgence = pd.read_csv('Ensemble_decrets_urgence.tsv', sep='\t')
corpus_urgence["date"] = pd.to_datetime(corpus_urgence["date"])
df = corpus_urgence.sort_values(by="date")
df_triee = df[df.date.isnull()]

#Creation d'une liste de tous les types de décrets pour récupérer les intéressants (suppression des messages non datés par exemple)
liste = []
for items in df_triee['type de décret'].unique():
    liste.append(items)
liste_mots_conservés = ['arrete', 'decret', 'articles', 'unknown', 'proclamation', 'declaration', 'Declaration', 'discours', 'Arrêté', 'Décret', 'Déclaration', 'Loi', 'Arrêtés', 'loi', 'acte', 'instruction', 'Proclamation', 'arreie', 'lois', 'decrete', 'acrete', 'decet', 'rapport', 'artete', 'arretes', 'atrete', 'resolution', 'adresse', 'liquidation', 'deliberation', 'autorisation', 'acceptation', 'nomination', 'Decret', 'article', 'Instruction','arrette', 'Arrete', 'avis', 'disposition','approbation','Article','Dispositions']

#tri en utilisant la liste des mots conservés prédéfinie 
df_triee2 = df_triee
tri_final = df_triee2.loc[df_triee2['type de décret'].isin(liste_mots_conservés)]
tri_final2 = tri_final[tri_final.date.isnull()]

#sauvegarde de cette liste de mots triés
tri_final2.to_csv('Erreurs_lois.tsv', sep='\t', encoding="utf-8")

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df[["date","ID"]])
    
#au total, 475 erreurs de dates

In [ ]:
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df[["date","ID"]])
    
#au total, 475 erreurs de dates

# Test avec un pas de 365 + Definition de fonction

In [9]:
def extract_jsonl(dataframe, nom_fichier):
    docs = nlp.pipe(dataframe.texte)
    ID_corpus = dataframe.ID
    with codecs.open(nom_fichier, 'w', encoding='utf8') as f:
        increment = 0
        for doc in docs:
            text = doc.text
            labels = []
            ID_2 = ID_corpus.iloc[increment]
            ID_2_count = len(ID_2)
            IDs = (ID_2)
            if (len(doc.ents) > 0):
                for ent in doc.ents:
                    labels.append([ent.start_char+ID_2_count, ent.end_char+ID_2_count, ent.label_])
            if (len(labels) > 0):
                sentence = {"text": IDs + text, "labels": labels}    
            else:
                sentence = {"text": IDs + text}
            json_string = json.dumps(sentence, ensure_ascii=False)
            increment += 1
            f.write(json_string)
            f.write("\n")
    f.close()
    print(nom_fichier, 'exporté')

In [6]:
corpus_urgence = pd.read_csv('Ensemble_decrets_urgence.tsv', sep='\t')
urgence_1 = corpus_urgence[0:365]
urgence_2 = corpus_urgence[366:750]
urgence_3 = corpus_urgence[751:1095]
urgence_4 = corpus_urgence[1096:1460]
urgence_5 = corpus_urgence[1461:1825]
urgence_6 = corpus_urgence[1826:2190]
urgence_7 = corpus_urgence[2191:2555]
urgence_8 = corpus_urgence[2556:2920]
urgence_9 = corpus_urgence[2921:3285]
urgence_10 = corpus_urgence[3286:3650]
urgence_11 = corpus_urgence[3651:3707]

### ATTENTION : l'entrée suivante a un grand coût d'exécution !
###  Prévoir environ 2 heures pour terminer l'exécution de la fonction pour tous les sous-corpus créés sur un ordinateur normal (4 coeur, 16go RAM)

In [ ]:
extract_jsonl(urgence_1, "urgence1.jsonl")
extract_jsonl(urgence_2, "urgence2.jsonl")
extract_jsonl(urgence_3, "urgence3.jsonl")
extract_jsonl(urgence_4, "urgence4.jsonl")
extract_jsonl(urgence_5, "urgence5.jsonl")
extract_jsonl(urgence_6, "urgence6.jsonl")
extract_jsonl(urgence_7, "urgence7.jsonl")
extract_jsonl(urgence_8, "urgence8.jsonl")
extract_jsonl(urgence_9, "urgence9.jsonl")
extract_jsonl(urgence_10, "urgenc10.jsonl")
extract_jsonl(urgence_11, "urgence11.jsonl")